In [1]:
from azureml.core import Experiment, Run, Workspace
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.85


In [2]:
#subscription_id = '6a0ec27b-1203-46fe-ade0-8c3003d5e4bc'
#resource_group  = 'MLServiceWrokspace'
#workspace_name  = 'MLServiceWrokspace'
#
#try:
#    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#    ws.write_config()
#    print('Library configuration succeeded')
#except:
#    print('Workspace not found')
import pandas as pd

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-classification-ccard-local'

experiment=Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T


,
Subscription ID,6a0ec27b-1203-46fe-ade0-8c3003d5e4bc
Workspace,MLServiceWrokspace
Resource Group,MLServiceWrokspace
Location,northeurope
Experiment Name,automl-classification-ccard-local


In [3]:
experiment_name = 'autoMLdemo5'

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import logging
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.explain.model._internal.explanation_client import ExplanationClient

irisds = load_iris()

iris = sns.load_dataset("iris")
type(iris)

pandas.core.frame.DataFrame

In [5]:
train, test = train_test_split(iris, test_size=0.2, random_state=42, shuffle=True)

In [6]:
##If we were not using 'preprocess = True' then we would have to convert to a numeric 

##Create a function to convert species to numeric
#def func(x):
#     if x == 'setosa':
#          return 1
#     elif x == 'versicolor':
#          return 2
#     elif x == 'virginica':
#          return 3

#from numpy import vectorize
#vfunc = vectorize(func) #Define a vectorized function which takes a nested sequence of objects or numpy arrays as inputs

#yfactTrain = vfunc(y_train)
#yfactTest = vfunc(y_test)

#yfactTrain = yfactTrain.reshape(len(yfactTrain))
#yfactTest = yfactTest.reshape(len(yfactTest))


In [7]:
#Make sure all the features are floats
#X_train = X_train.astype(np.float)
#X_test = X_test.astype(np.float)

In [6]:
train.head()

,sepal_length,sepal_width,petal_length,petal_width,species
22,4.60,3.60,1.00,0.20,setosa
15,5.70,4.40,1.50,0.40,setosa
65,6.70,3.10,4.40,1.40,versicolor
11,4.80,3.40,1.60,0.20,setosa
42,4.40,3.20,1.30,0.20,setosa


In [9]:
test.head()

,sepal_length,sepal_width,petal_length,petal_width,species
73,6.1,2.8,4.7,1.2,versicolor
18,5.7,3.8,1.7,0.3,setosa
118,7.7,2.6,6.9,2.3,virginica
78,6.0,2.9,4.5,1.5,versicolor
76,6.8,2.8,4.8,1.4,versicolor


In [7]:
automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": 'accuracy',
    #"experiment_timeout_hours": 0.25, # This is a time limit for testing purposes, remove it for real use cases, this will drastically limit ability to find the best model possible
    "verbosity": logging.INFO,
    "enable_stack_ensemble": False
}

label_column_name = 'species'

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             training_data = train,
                             label_column_name = label_column_name,
                             iterations = 6,
                             #blacklist_models = ['kNN','LinearSVM'],
                             **automl_settings
                            )

In [8]:
from azureml.core.experiment import Experiment
experiment=Experiment(ws, experiment_name)
local_run = experiment.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_a2f3ac74-24a5-447e-ae63-e3f040551d8c

Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Classes are balanced in the training data.

TYPE:         Missing values imputation
STATUS:       PASSED
DESCRIPTION:  There were no missing values found in the training data.

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected

In [9]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
#Breakdown of the results
children = list(local_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

import pandas as pd
rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
AUC_macro,0.99,0.96,0.99,0.93,0.98,0.93,0.96,0.86,0.99,0.99,...,0.97,0.50,0.98,0.99,1.00,1.00,0.50,0.99,1.00,nan
AUC_micro,0.99,0.97,0.99,0.92,0.97,0.95,0.97,0.94,0.99,0.99,...,0.98,0.49,0.98,0.99,1.00,1.00,0.45,1.00,1.00,nan
AUC_weighted,0.99,0.95,0.99,0.93,0.97,0.92,0.96,0.86,0.99,0.99,...,0.97,0.50,0.98,0.99,1.00,1.00,0.50,0.99,1.00,nan
accuracy,0.93,0.88,0.91,0.69,0.92,0.81,0.89,0.96,0.92,0.92,...,0.92,0.31,0.92,0.94,0.95,0.95,0.28,0.93,0.95,nan
average_precision_score_macro,0.98,0.93,0.98,0.86,0.97,0.88,0.94,0.80,0.98,0.98,...,0.95,0.33,0.96,0.98,1.00,0.99,0.33,0.99,0.99,nan
average_precision_score_micro,0.98,0.94,0.98,0.87,0.95,0.91,0.95,0.90,0.98,0.98,...,0.96,0.33,0.97,0.99,1.00,1.00,0.31,0.99,0.99,nan
average_precision_score_weighted,0.98,0.92,0.98,0.86,0.96,0.88,0.94,0.79,0.98,0.98,...,0.95,0.34,0.96,0.98,1.00,0.99,0.34,0.99,0.99,nan
balanced_accuracy,0.93,0.90,0.91,0.70,0.92,0.82,0.89,0.96,0.91,0.93,...,0.91,0.33,0.91,0.94,0.95,0.95,0.33,0.93,0.95,nan
f1_score_macro,0.93,0.89,0.91,0.62,0.92,0.81,0.89,0.96,0.91,0.92,...,0.91,0.16,0.91,0.94,0.95,0.95,0.15,0.93,0.95,nan
f1_score_micro,0.93,0.88,0.91,0.69,0.92,0.81,0.89,0.96,0.92,0.92,...,0.92,0.31,0.92,0.94,0.95,0.95,0.28,0.93,0.95,nan


In [11]:
#Get the best model
best_run, fitted_model = local_run.get_output()


In [13]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
autoMLdemo5,AutoML_a2f3ac74-24a5-447e-ae63-e3f040551d8c_26,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [28]:
#Test with the testing data

from sklearn.metrics import confusion_matrix, roc_curve, auc

y_pred = (fitted_model.predict(test.iloc[:,0:4]))

print(accuracy_score(test.iloc[:,4], y_pred))

cm = confusion_matrix(test.iloc[:,4], y_pred)
print(cm)


0.9666666666666667
[[10  0  0]
 [ 0  8  1]
 [ 0  0 11]]
